<a href="https://colab.research.google.com/github/ninjutsucoding/Home_Sales/blob/main/Home_Sales_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install findspark

In [2]:
# Import findspark and initialize.
import findspark
findspark.init()

In [35]:
# Import packages
from pyspark.sql import SparkSession
import time

from pyspark.sql.functions import year

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [5]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=",", inferSchema=True, header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('home_sales')

In [23]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
SELECT YEAR(date) AS year,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR(date)
ORDER BY year;
""").show()


+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [26]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
SELECT date_built AS year_built,
  ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3
GROUP BY date_built
ORDER BY date_built;
""").show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [30]:
#5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
SELECT date_built AS year_built,
     ROUND(AVG(price), 2) AS average_price
FROM home_sales
WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 3 AND sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built;
""").show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|     774005.0|
|      2011|     693529.5|
|      2012|    931164.33|
|      2013|    674985.33|
|      2014|    812559.25|
|      2015|     663862.8|
|      2016|     893094.5|
|      2017|    686528.17|
+----------+-------------+



In [36]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
start_time = time.time()

spark.sql("""
SELECT view,
ROUND(AVG(price / view), 2) as avg_price_per_view
FROM home_sales
WHERE price >= 35000
GROUP BY view
ORDER BY view DESC;
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|avg_price_per_view|
+----+------------------+
| 100|          10266.69|
|  99|          10719.21|
|  98|          10752.44|
|  97|          11639.59|
|  96|          10602.25|
|  95|          11098.16|
|  94|          10995.07|
|  93|          11032.32|
|  92|          10547.85|
|  91|           12498.6|
|  90|          11807.27|
|  89|          12447.63|
|  88|          11724.08|
|  87|          12325.12|
|  86|          12447.03|
|  85|          12427.49|
|  84|          13300.39|
|  83|          12457.42|
|  82|          12969.49|
|  81|          13005.84|
+----+------------------+
only showing top 20 rows

--- 0.7308008670806885 seconds ---


In [37]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

DataFrame[]

In [38]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [39]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
SELECT view,
ROUND(AVG(price / view), 2) as avg_price_per_view
FROM home_sales
WHERE price >= 35000
GROUP BY view
ORDER BY view DESC;
""").show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------------+
|view|avg_price_per_view|
+----+------------------+
| 100|          10266.69|
|  99|          10719.21|
|  98|          10752.44|
|  97|          11639.59|
|  96|          10602.25|
|  95|          11098.16|
|  94|          10995.07|
|  93|          11032.32|
|  92|          10547.85|
|  91|           12498.6|
|  90|          11807.27|
|  89|          12447.63|
|  88|          11724.08|
|  87|          12325.12|
|  86|          12447.03|
|  85|          12427.49|
|  84|          13300.39|
|  83|          12457.42|
|  82|          12969.49|
|  81|          13005.84|
+----+------------------+
only showing top 20 rows

--- 0.4360358715057373 seconds ---


In [40]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('date_built_partitioned')

In [41]:
# 11. Read the formatted parquet data.
p_df = spark.read.parquet('date_built_partitioned')

In [42]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_date_built_p')

In [43]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
SELECT view,
ROUND(AVG(price / view), 2) as avg_price_per_view
FROM p_date_built_p
WHERE price >= 35000
GROUP BY view
ORDER BY view DESC;
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------------+
|view|avg_price_per_view|
+----+------------------+
| 100|           10266.7|
|  99|          10719.21|
|  98|          10752.44|
|  97|          11639.59|
|  96|          10602.25|
|  95|          11098.16|
|  94|          10995.07|
|  93|          11032.32|
|  92|          10547.85|
|  91|           12498.6|
|  90|          11807.27|
|  89|          12447.63|
|  88|          11724.08|
|  87|          12325.12|
|  86|          12447.03|
|  85|          12427.49|
|  84|          13300.39|
|  83|          12457.42|
|  82|          12969.49|
|  81|          13005.84|
+----+------------------+
only showing top 20 rows

--- 1.0651073455810547 seconds ---


In [44]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('home_sales')

In [45]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False